In [40]:
from mesa import Model
from mesa import Agent
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random
import numpy as np
import pickle
import cv2

from a_star_pathfinding import a_star
from vision import lines
from vision import transform

In [41]:
def euclidean(pos_a, pos_b):
    return ((pos_a[0] - pos_b[0])**2 + (pos_a[1] - pos_b[1])**2)**.5

In [214]:
class AmongUs(Model):
    def __init__(self, map_name, n_crew, n_impo):
        super().__init__()

        # generate grid
        self.height = 138
        self.width = 242
        self.grid = MultiGrid(self.width, self.height, torus=True)

        # generate map
        self.generate_map(map_name)
        
        # generate tasks
        self.generate_tasks(map_name)
        
        # load in vision dictionary
        vision_file = open("vision_dict.pkl", "rb")
        vision_dict = pickle.load(vision_file)
        
        # create schedulers for agents
        self.schedule_Crewmate = RandomActivation(self)
        self.schedule_Imposter = RandomActivation(self)
        
        # initialize agents
        self.n_crew = n_crew
        self.n_impo = n_impo
        self.spawn_players()

    def generate_map(self, map_name):
        hard_walls = np.load(f'{map_name}/hardwalls.npy')
        vents = np.load(f'{map_name}/vents.npy', allow_pickle=True)
        
        for coord in hard_walls:
            coord[1] = self.height - 1 - coord[1]
            self.new_agent(Wall, tuple(coord))

        self.vents_dict = {}
        self.vents = []

        for connection in vents:
            connection = connection.tolist()
            self.vents_dict[tuple(connection[0])] = tuple(connection[1])

        for coord in self.vents_dict:
            self.new_agent(Vent, tuple(coord))

    def generate_tasks(self, map_name):
        # load the image + coordinates of the short tasks
        short_tasks = np.load(f'{map_name}/shorttasks.npy', allow_pickle=True)
        self.short_tasks = short_tasks
        self.tasks = []

        # load all the short_tasks into the map (all possible task locations), first create tuples
        for i in range(len(short_tasks)):
            if i == 3:
                self.new_agent(ShortTask, tuple(short_tasks[i][0][0]))
                for parttwo in range(8):
                    self.new_agent(ShortTask, tuple(short_tasks[i][1][parttwo]))
            elif i == 6:
                for partone in range(5):
                    self.new_agent(ShortTask, tuple(short_tasks[i][0][partone]))
                self.new_agent(ShortTask, tuple(short_tasks[i][1][0]))
        
            else:
                self.new_agent(ShortTask, tuple(short_tasks[i][0]))

    def spawn_players(self):
        # generate crewmates
        for _ in range(self.n_crew):
            self.new_agent(Crewmate, (125, 100))

        # generate imposters
        for _ in range(self.n_impo):
            self.new_agent(Imposter, (125, 100))

    def new_agent(self, agent_type, pos):
        '''
        Method that creates a new agent, and adds it to the correct scheduler.
        '''
        agent = agent_type(self.next_id(), self, pos)

        self.grid.place_agent(agent, pos)

        if agent_type == Crewmate or agent_type == Imposter:
            getattr(self, f'schedule_{agent_type.__name__}').add(agent)

        if agent_type == Vent:
            self.vents.append(agent)
        
        if agent_type == ShortTask:
            print(agent)
            self.tasks.append(agent)

    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''

        self.schedule_Crewmate.step()
        self.schedule_Imposter.step()

        # self.datacollector.collect(self)

    def play_match(self, step_count=200):
        '''
        Method that runs the model for a specific amount of steps.
        '''
        for i in range(step_count):
            print(i)
            self.step()


In [220]:
class Crewmate(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goallist = []
        self.injob = 0
        
    def make_tasks(self):
        short_tasks = self.model.short_tasks
        print(short_tasks)
        
        # AANPASSING
        # set 4 random tasks for Crewmate to execute (can still be same task multiple times)
        self.goallist = random.sample(list(short_tasks), 4)
        
        # if it is a task with multiple locations/second part, replace by one of the coordinates from the first list 
        self.goallist = [[random.choice(self.goallist[index][0])] if len(x)>1 else x for index,x in enumerate(self.goallist)]
        
        # Turn list into list of tuples ### GEEFT SOMS EEN ERROR 
        self.goallist = [tuple(self.goallist[index][0]) for index,y in enumerate(self.goallist)]
        
    def find_path(self):
        if len(self.goallist) == 0:
            self.make_tasks()
        
        possiblepaths = []
        for t in self.goallist:
            lst = list(t)
            lst[1] = 138 - 1 - lst[1]
            t = tuple(lst)
            
            possiblepaths.append(a_star(self.pos, t, self.model.grid)[0])
            
        self.path = self.path + min(possiblepaths)
        print(self.path)

    def step(self):
        if self.path == []:
            ##fix coordinaten moet er nog uit
            lst = list(self.pos)
            lst[1] = 138 - 1 - lst[1]
            t = tuple(lst)
            
            if t in self.goallist:
                self.goallist.remove(t)
                self.injob = random.randint(5, 15)
                print(self.path)
                
            # if Task was task 3, add second part of task 3 to the task list
            if self.pos == tuple(self.model.short_tasks[3][0][0]):
                self.goallist.append(tuple(random.choice(self.model.short_tasks[3][1][0])))
            
            # if Task was task 6, add second part of task 6 to the task list
            elif self.pos in self.model.short_tasks[6][0]:
                self.goallist.append(tuple(self.model.short_tasks[6][1][0]))
            
            self.find_path()

        elif self.path != []:
            if self.injob > 0:
                self.injob -= 1
            else:
                print('currrent goal', self.path[0])
                self.model.grid.move_agent(self, self.path.pop())            
            print('current place', self.pos)
            
class Imposter(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goals = [(85, 35), (140, 110), (125, 90)]
        self.vents_enabled = True
    
    def find_path(self):
        if len(self.goals) > 0:
            random.shuffle(self.goals)
            goal = self.goals.pop()
            best_path, best_score = a_star(self.pos, goal, self.model.grid)

            if self.vents_enabled:

                for vent in self.model.vents:
                    vent_in = vent.pos_a
                    vent_out = vent.pos_b
                    guess_vent_path = euclidean(self.pos, vent_in) + euclidean(vent_out, goal)

                    if guess_vent_path < best_score:
                        vent_path_in, vent_score_in = a_star(self.pos, vent_in, self.model.grid)
                        vent_path_out, vent_score_out = a_star(vent_out, goal, self.model.grid)
                        vent_path = vent_path_out + vent_path_in
                        vent_score = vent_score_in + vent_score_out
    
                        if vent_score < best_score:
                            best_score = vent_score
                            best_path = vent_path

            self.path = best_path

    def step(self):
        if self.path == []:
            self.find_path()

#        grid = self.model.grid
#        neighborhood = grid.get_neighborhood(self.pos, True)
#
#        neighbors = self.model.grid.get_neighbors(self.pos, True)
#
#        if len(neighbors) > 0:
#            for neighbor in neighbors:
#                if type(neighbor) == Wall and neighbor.pos in neighborhood:
#                    neighborhood.remove(neighbor.pos)

        if self.path != []:
            self.model.grid.move_agent(self, self.path.pop())               
                        
class Wall(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Obstruction(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Vent(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos_a = pos
        self.pos_b = self.model.vents_dict[pos]

class ShortTask(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

In [221]:
amongUs = AmongUs('the_skeld', 1, 0)

amongUs.play_match()

0
[array([[101,  82]], dtype=int64) array([[237,  54]], dtype=int64)
 array([[167,  52]], dtype=int64)
 list([array([[83, 79]], dtype=int64), array([[ 38,  19],
       [201,  25],
       [226,  51],
       [182,  52],
       [ 20,  55],
       [ 33,  90],
       [197,  95],
       [170, 119]], dtype=int64)])
 array([[239,  62]], dtype=int64) array([[ 7, 49]], dtype=int64)
 list([array([[154,   8],
       [184,  14],
       [232,  51],
       [ 78,  79],
       [158, 119]], dtype=int64), array([[149,  74]], dtype=int64)])
 array([[177, 117]], dtype=int64)]
[(78, 58), (79, 58), (80, 58), (81, 58), (82, 58), (83, 57), (84, 56), (85, 55), (86, 54), (87, 53), (88, 52), (89, 51), (90, 50), (91, 49), (91, 48), (91, 47), (91, 46), (91, 45), (91, 44), (91, 43), (91, 42), (91, 41), (91, 40), (91, 39), (91, 38), (90, 37), (89, 36), (88, 35), (87, 34), (86, 33), (85, 32), (84, 31), (84, 30), (84, 29), (83, 28), (83, 27), (84, 26), (85, 26), (86, 26), (87, 26), (88, 26), (89, 26), (90, 26), (91, 26

[(83, 58), (82, 58), (81, 58), (80, 58), (79, 58)]
133
current place (78, 58)
134
current place (78, 58)
135
current place (78, 58)
136
current place (78, 58)
137
current place (78, 58)
138
current place (78, 58)
139
current place (78, 58)
140
current place (78, 58)
141
current place (78, 58)
142
current place (78, 58)
143
current place (78, 58)
144
current place (78, 58)
145
current place (78, 58)
146
current place (78, 58)
147
currrent goal (83, 58)
current place (79, 58)
148
currrent goal (83, 58)
current place (80, 58)
149
currrent goal (83, 58)
current place (81, 58)
150
currrent goal (83, 58)
current place (82, 58)
151
currrent goal (83, 58)
current place (83, 58)
152
[]
[(101, 55), (100, 55), (99, 55), (98, 55), (97, 55), (96, 55), (95, 55), (94, 55), (93, 55), (92, 55), (91, 55), (90, 55), (89, 55), (88, 55), (87, 55), (86, 55), (85, 55), (84, 56), (84, 57)]
153
current place (83, 58)
154
current place (83, 58)
155
current place (83, 58)
156
current place (83, 58)
157
current p

In [195]:
s = [1]

s = s + [1, 2, 3]


print(s)

[1, 1, 2, 3]
